In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
print(sys.path)

['/home/hieutt/UniCon/notebooks', '/home/hieutt/miniconda3/envs/torchtf/lib/python39.zip', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/lib-dynload', '', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/site-packages', '../']


In [2]:
import argparse
import math
import os
import random
import sys
import time
import numpy as np
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import pprint
from torchvision import transforms
from dataset import CANDatasetEnet as CANDataset
from losses import SupConLoss, UniConLoss
from networks.efficient_net import ConEfficientNet, LinearClassifier
from networks.efficient_net_b0 import ConEfficientNet as E_Unicon_B0, LinearClassifier as L_Unicon_B0
from util import TwoCropTransform, AverageMeter, AddGaussianNoise
from util import warmup_learning_rate
from util import get_universum
from util import save_model ,load_checkpoint, accuracy
# from networks.classifier import LinearClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score

In [3]:
from collections import namedtuple

# Định nghĩa cấu trúc `Opt` với các trường cần thiết
Opt = namedtuple('Opt', ['lamda', 'mix', 'device', 'data_folder', 'num_workers', 'batch_size'])

# Tạo đối tượng opt
opt = Opt(
    lamda=0.5,
    mix='mixup',
    device='cuda',
    data_folder='../data/can-ml/2017-subaru-forester/preprocessed/size_64_10/TFRecord_w64_s32/2',
    num_workers=8,
    batch_size=64
)

# Truy cập các tham số
print(opt.lamda)
print(opt.device)


0.5
cuda


In [4]:
# Hàm này sẽ lấy một subset nhỏ từ dataset
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import transforms

def get_subset_loader(dataset, subset_ratio=0.1, batch_size=32):
    subset_size = int(len(dataset) * subset_ratio)  # Tính số lượng phần tử của subset
    indices = torch.randperm(len(dataset)).tolist()[:subset_size]  # Chọn ngẫu nhiên các chỉ số
    subset = Subset(dataset, indices)  # Tạo Subset từ dataset gốc
    
    # Tạo DataLoader từ subset
    subset_loader = DataLoader(subset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)  # Chỉnh batch_size, num_workers theo yêu cầu
    return subset_loader

In [5]:
def set_loader(opt, batch_size, subset_ratio=0.1):
    mean = (0.5, 0.5, 0.5)
    std = (0.5, 0.5, 0.5)
    normalize = transforms.Normalize(mean=mean, std=std)
    transform = transforms.Compose([normalize])
    train_transform = transforms.Compose([
        transforms.RandomApply([AddGaussianNoise(0., 0.05)], p=0.5),
        transforms.RandomErasing(p=0.3, scale=(0.02, 0.15)),
        transforms.Normalize(mean=mean, std=std)
    ])
    
    # Khởi tạo dataset
    train_dataset = CANDataset(root_dir=opt.data_folder, window_size=32, is_train=True, transform=TwoCropTransform(train_transform))
    test_dataset = CANDataset(root_dir=opt.data_folder, window_size=32, is_train=False, transform=transform)

    # Lấy subset nhỏ của dataset
    train_loader = get_subset_loader(train_dataset, subset_ratio=subset_ratio, batch_size=batch_size)  
    test_loader = get_subset_loader(test_dataset, subset_ratio=subset_ratio, batch_size=batch_size)

    return train_loader, test_loader

In [6]:

def get_predict(outputs):
    _, pred = outputs.topk(1, 1, True, True)
    pred = pred.t().cpu().numpy().squeeze(0)
    return pred


In [7]:
def train_classifier(train_loader, model, classifier, criterion, optimizer, epoch, opt):
    model.eval()
    classifier.train()

    losses = AverageMeter()
    accs = AverageMeter()

    # Start the training loop
    end = time.time()
    for idx, (images, labels) in enumerate(train_loader):
        images = images[0]
        # if torch.cuda.is_available():
        #     images = images.cuda(non_blocking=True)
        #     labels = labels.cuda(non_blocking=True)
        images = images.to(opt.device, non_blocking=True)
        labels = labels.to(opt.device, non_blocking=True)
        bsz = labels.size(0)  # Batch size

        # Extract features from the pre-trained model in evaluation mode
        with torch.no_grad():
            features = model.encoder(images)

        # Forward pass through the classifier
        output = classifier(features.detach())

        # Compute loss
        loss = criterion(output, labels)
        losses.update(loss.item(), bsz)

        # Compute accuracy
        acc = accuracy(output, labels, topk=(1,))
        accs.update(acc[0].item(), bsz)

        # Backward pass and optimization step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [8]:
def validate(val_loader, model, classifier, criterion, opt):
    model.eval()
    classifier.eval()
    
    losses = AverageMeter()
    total_pred = np.array([], dtype=int)
    total_label = np.array([], dtype=int) 
    
    with torch.no_grad():
        for images, labels in tqdm(val_loader): 
            # if torch.cuda.is_available():
            #     images = images.cuda(non_blocking=True)
            #     labels = labels.cuda(non_blocking=True)
            images = images.to(opt.device, non_blocking=True)
            labels = labels.to(opt.device, non_blocking=True)
            features = model.encoder(images)
            outputs = classifier(features)

            bsz = labels.size(0)
            loss = criterion(outputs, labels)
            losses.update(loss.item(), bsz)
            
            pred = get_predict(outputs)

            if isinstance(pred, torch.Tensor):
                pred = pred.cpu().numpy()
            
            if isinstance(labels, torch.Tensor):
                labels = labels.cpu().numpy()

            total_pred = np.concatenate((total_pred, pred), axis=0)
            total_label = np.concatenate((total_label, labels), axis=0)
    
    acc = accuracy_score(total_label, total_pred)
    acc = acc * 100
    f1 = f1_score(total_label, total_pred, average='weighted')
    
    return acc, f1

In [9]:
def change_state_dict(state_dict):
    """
    Because state dict in distributed GPU is different
    """
    new_state_dict = {}
    for k, v in state_dict.items():
        k = k.replace("module.", "")
        new_state_dict[k] = v
    return new_state_dict

In [10]:
# model = ConEfficientNet(embedding_dim=1792, feat_dim=128, head='mlp', pretrained=False)
# classifier = LinearClassifier(input_dim=1792, num_classes=10)

model = E_Unicon_B0(embedding_dim=1280, feat_dim=128, head='mlp', pretrained=False)
classifier = L_Unicon_B0(input_dim=1280, num_classes=10)


In [10]:
# save_path = '../save/CAN-ML_models/UniCon/UniCon_CAN-ML_efficient-net_lr_0.05_decay_0.0001_bsz_64_temp_0.07_mixup_lambda_0.5_trial_can_ml_con_enet_b4_64_cosine_warm'
# ckpt_epoch = 158
# # save_path = '../save/CAN-ML_models/UniCon/UniCon_CAN-ML_resnet50_lr_0.05_decay_0.0001_bsz_64_temp_0.07_mixup_lambda_0.5_trial_can_ml_uni_resnet_cosine_warm/'
# # ckpt_epoch = 37
# model_path = f'{save_path}/last.pth'
# model_path = f'{save_path}/ckpt_epoch_{ckpt_epoch}.pth'
# ckpt = torch.load(model_path, weights_only=False)
# state_dict = ckpt['model']
# state_dict = change_state_dict(state_dict)
# model.load_state_dict(state_dict=state_dict)

<All keys matched successfully>

In [11]:
from torch import nn
def train_encoder(train_loader, learning_rate, criterion, optimizer):
  model.train()

  for images, labels in train_loader:
      image1, image2 = images[0], images[1]
      images = torch.cat([image1, image2], dim=0)
      images = images.to('cuda')
      labels = labels.to('cuda')
      bsz = labels.shape[0]

      universum = get_universum(images, labels, opt)
      uni_features = model(universum)

      features = model(images)
      f1, f2 = torch.split(features, [bsz, bsz], dim=0)
      features = torch.cat([f1.unsqueeze(1), f2.unsqueeze(1)], dim=1)

      loss = criterion(features, uni_features, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

In [15]:
import optuna
import torch.optim as optim
from torch import nn
from sklearn.metrics import accuracy_score

def objective(trial):
    # 1. Tối ưu các siêu tham số
    torch.cuda.empty_cache()
    # learning_rate = trial.suggest_float('learning_rate', 1e-5, 0.001, log=True)
    # new_batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256])
    learning_rate = 0.05  # Ví dụ, bạn muốn dùng giá trị này cho tất cả các trial

    # Sử dụng Optuna để chọn batch_size
    batch_sizes = [8, 16, 32, 64, 128]
    new_batch_size = batch_sizes[trial.number % len(batch_sizes)]

    train_loader, val_loader = set_loader(opt, new_batch_size)

    model.to(opt.device)
    classifier.to(opt.device)
    # model = nn.DataParallel(model, device_ids=[0, 1])  # Chạy trên 2 GPU
    # classifier = nn.DataParallel(classifier, device_ids=[0, 1])  # Nếu classifier cũng cần
    # 4. Loss và Optimizer (chỉ tối ưu classifier)
    criterion = UniConLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-4)
    criterion_classifier = torch.nn.CrossEntropyLoss()
    optimizer_classifier = torch.optim.SGD(classifier.parameters(), lr=0.01, momentum=0.9, weight_decay=0)

    step = 0
    for epoch in range(5):
        train_encoder(train_loader, learning_rate, criterion, optimizer)
        train_classifier(
            train_loader, model, classifier, criterion_classifier, optimizer_classifier, epoch, opt
        )
        torch.cuda.empty_cache()
    acc, f1 = validate(
        val_loader, model, classifier, criterion_classifier, opt
    )

    print(f"[Trial {trial.number}] acc: {acc:.2f}, f1: {f1} lr: {learning_rate}, batch: {new_batch_size}")

    return f1  # F1 để Optuna tối ưu



In [13]:
storage="sqlite:///db.sqlite3",  # Specify the storage URL here.
study_name="hyper-parameter",  # Unique identifier of the study.

In [16]:
# Tạo một study và bắt đầu tối ưu hóa siêu tham số
study = optuna.create_study(direction='maximize')
# Chạy tối ưu hóa với 100 lần thử nghiệm
study.optimize(objective, n_trials=4)
# In kết quả tối ưu hóa
print("Best trial:")
print(f"  F1: {study.best_trial.value}")
print(f"  Params: {study.best_trial.params}")

[I 2025-03-28 09:57:46,881] A new study created in memory with name: no-name-cb78185f-b8a2-4de2-9639-607013860789
100%|██████████| 328/328 [00:04<00:00, 72.32it/s]
[I 2025-03-28 10:04:34,924] Trial 0 finished with value: 0.6352054339687556 and parameters: {}. Best is trial 0 with value: 0.6352054339687556.


[Trial 0] acc: 62.80, f1: 0.6352054339687556 lr: 0.05, batch: 8


100%|██████████| 164/164 [00:02<00:00, 66.38it/s]
[I 2025-03-28 10:08:09,516] Trial 1 finished with value: 0.7101599570479864 and parameters: {}. Best is trial 1 with value: 0.7101599570479864.


[Trial 1] acc: 77.38, f1: 0.7101599570479864 lr: 0.05, batch: 16


100%|██████████| 82/82 [00:01<00:00, 41.26it/s]
[I 2025-03-28 10:10:15,543] Trial 2 finished with value: 0.7061265245998999 and parameters: {}. Best is trial 1 with value: 0.7101599570479864.


[Trial 2] acc: 79.32, f1: 0.7061265245998999 lr: 0.05, batch: 32


100%|██████████| 41/41 [00:02<00:00, 20.32it/s]
[I 2025-03-28 10:11:49,182] Trial 3 finished with value: 0.7331108743493506 and parameters: {}. Best is trial 3 with value: 0.7331108743493506.


[Trial 3] acc: 80.54, f1: 0.7331108743493506 lr: 0.05, batch: 64
Best trial:
  F1: 0.7331108743493506
  Params: {}
